In [ ]:
# Enhanced AutoEncoder Training for Complete Dataset
# This notebook creates a comprehensive AutoEncoder trained on the entire enhanced dataset 
# with proper overfitting prevention and progress visualization.

# Import Required Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print(f"📊 PyTorch version: {torch.__version__}")
print(f"🎯 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🚀 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("?️ Using CPU for training")

In [ ]:
# Load and Prepare Enhanced Dataset
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '..'))

print("🔄 Loading real telecom data...")

# Load original data
data_file = os.path.join('..', 'data', 'AD_data_10KPI.csv')
if not os.path.exists(data_file):
    data_file = os.path.join('..', 'AD_data_10KPI.csv')

if os.path.exists(data_file):
    df_original = pd.read_csv(data_file)
    print(f"📈 Original data shape: {df_original.shape}")
    print("✅ Data loaded successfully!")
    print("\nFirst few rows:")
    print(df_original.head())
else:
    print(f"❌ Data file not found at {data_file}")
    print("Creating sample data for demonstration...")
    
    # Create sample data for demonstration
    np.random.seed(42)
    n_samples = 1000
    n_sites = 20
    
    data = {
        'Date': pd.date_range('2024-01-01', periods=n_samples, freq='H'),
        'Site_ID': np.random.choice([f'Site_{i:03d}' for i in range(1, n_sites+1)], n_samples),
        'RSRP': np.random.normal(-85, 10, n_samples),
        'SINR': np.random.normal(15, 5, n_samples),
        'DL_Throughput': np.random.lognormal(3, 0.5, n_samples),
        'UL_Throughput': np.random.lognormal(2, 0.5, n_samples),
        'CPU_Utilization': np.random.beta(2, 5, n_samples) * 100,
        'Active_Users': np.random.poisson(50, n_samples),
        'RTT': np.random.gamma(2, 5, n_samples),
        'Packet_Loss': np.random.beta(1, 10, n_samples) * 5,
        'Call_Drop_Rate': np.random.beta(1, 20, n_samples) * 2,
        'Handover_Success_Rate': np.random.beta(8, 2, n_samples) * 100
    }
    
    df_original = pd.DataFrame(data)
    print(f"📈 Sample data created with shape: {df_original.shape}")
    print("✅ Sample data ready for training!")
    print("\nFirst few rows:")
    print(df_original.head())

In [ ]:
# Enhanced Feature Engineering
print("🛠️ Performing feature engineering...")

# Define KPI columns
kpi_columns = ['RSRP', 'SINR', 'DL_Throughput', 'UL_Throughput', 'CPU_Utilization', 
               'Active_Users', 'RTT', 'Packet_Loss', 'Call_Drop_Rate', 'Handover_Success_Rate']

# Start with original KPIs
df_enhanced = df_original.copy()

# Sort by Site_ID and Date for time-series features
if 'Date' in df_enhanced.columns:
    df_enhanced['Date'] = pd.to_datetime(df_enhanced['Date'])
    df_enhanced = df_enhanced.sort_values(['Site_ID', 'Date'])

print(f"📊 Starting with {len(kpi_columns)} original KPIs")

# 1. Rolling statistics (moving averages, std)
rolling_windows = [3, 6, 12, 24]  # Hours
for window in rolling_windows:
    for kpi in kpi_columns:
        if kpi in df_enhanced.columns:
            # Rolling mean
            df_enhanced[f'{kpi}_rolling_mean_{window}h'] = (
                df_enhanced.groupby('Site_ID')[kpi]
                .rolling(window=window, min_periods=1)
                .mean()
                .reset_index(level=0, drop=True)
            )
            
            # Rolling std
            df_enhanced[f'{kpi}_rolling_std_{window}h'] = (
                df_enhanced.groupby('Site_ID')[kpi]
                .rolling(window=window, min_periods=1)
                .std()
                .reset_index(level=0, drop=True)
            )

print(f"✅ Added rolling statistics for {len(rolling_windows)} windows")

# 2. Lag features
lag_periods = [1, 3, 6]  # Hours
for lag in lag_periods:
    for kpi in kpi_columns:
        if kpi in df_enhanced.columns:
            df_enhanced[f'{kpi}_lag_{lag}h'] = (
                df_enhanced.groupby('Site_ID')[kpi]
                .shift(lag)
            )

print(f"✅ Added lag features for {len(lag_periods)} periods")

# 3. Rate of change features
for kpi in kpi_columns:
    if kpi in df_enhanced.columns:
        df_enhanced[f'{kpi}_rate_of_change'] = (
            df_enhanced.groupby('Site_ID')[kpi]
            .pct_change()
        )

print("✅ Added rate of change features")

# 4. Cross-KPI ratios and interactions
ratio_pairs = [
    ('DL_Throughput', 'UL_Throughput'),
    ('RSRP', 'SINR'),
    ('Active_Users', 'CPU_Utilization'),
    ('Call_Drop_Rate', 'Handover_Success_Rate')
]

for kpi1, kpi2 in ratio_pairs:
    if kpi1 in df_enhanced.columns and kpi2 in df_enhanced.columns:
        # Ratio
        df_enhanced[f'{kpi1}_{kpi2}_ratio'] = (
            df_enhanced[kpi1] / (df_enhanced[kpi2] + 1e-8)  # Add small epsilon to avoid division by zero
        )
        
        # Product
        df_enhanced[f'{kpi1}_{kpi2}_product'] = (
            df_enhanced[kpi1] * df_enhanced[kpi2]
        )

print(f"✅ Added cross-KPI features for {len(ratio_pairs)} pairs")

# 5. Time-based features
if 'Date' in df_enhanced.columns:
    df_enhanced['hour'] = df_enhanced['Date'].dt.hour
    df_enhanced['day_of_week'] = df_enhanced['Date'].dt.dayofweek
    df_enhanced['is_weekend'] = (df_enhanced['day_of_week'] >= 5).astype(int)
    df_enhanced['is_peak_hour'] = ((df_enhanced['hour'] >= 8) & (df_enhanced['hour'] <= 18)).astype(int)
    
    print("✅ Added temporal features")

# 6. Statistical aggregations per site
site_stats = df_enhanced.groupby('Site_ID')[kpi_columns].agg(['mean', 'std']).reset_index()
site_stats.columns = ['Site_ID'] + [f'site_{col[0]}_{col[1]}' for col in site_stats.columns[1:]]

df_enhanced = df_enhanced.merge(site_stats, on='Site_ID', how='left')
print("✅ Added site-level statistical features")

# Remove non-numeric columns for training
numeric_columns = df_enhanced.select_dtypes(include=[np.number]).columns
df_enhanced_numeric = df_enhanced[numeric_columns].copy()

# Fill any NaN values with forward fill, then backward fill, then 0
df_enhanced_numeric = df_enhanced_numeric.fillna(method='ffill').fillna(method='bfill').fillna(0)

print(f"\n📈 Feature engineering complete!")
print(f"🔍 Original features: {len(kpi_columns)}")
print(f"✨ Enhanced features: {len(df_enhanced_numeric.columns)}")
print(f"📊 Feature expansion: {len(kpi_columns)} → {len(df_enhanced_numeric.columns)} features")
print(f"🎯 Added features: {len(df_enhanced_numeric.columns) - len(kpi_columns)}")

print(f"\n📋 Feature categories:")
print(f"  • Original KPIs: {len(kpi_columns)}")
print(f"  • Rolling statistics: {len([c for c in df_enhanced_numeric.columns if 'rolling' in c])}")
print(f"  • Lag features: {len([c for c in df_enhanced_numeric.columns if 'lag' in c])}")
print(f"  • Rate of change: {len([c for c in df_enhanced_numeric.columns if 'rate_of_change' in c])}")
print(f"  • Cross-KPI features: {len([c for c in df_enhanced_numeric.columns if ('_ratio' in c or '_product' in c)])}")
print(f"  • Temporal features: {len([c for c in df_enhanced_numeric.columns if c in ['hour', 'day_of_week', 'is_weekend', 'is_peak_hour']])}")
print(f"  • Site statistics: {len([c for c in df_enhanced_numeric.columns if 'site_' in c])}")

# Display sample of enhanced data
print(f"\n🔍 Sample of enhanced data:")
print(df_enhanced_numeric.head())

In [ ]:
# Enhanced AutoEncoder Architecture
class EnhancedAutoEncoder(nn.Module):
    """
    Enhanced AutoEncoder with regularization and better architecture
    for multi-KPI anomaly detection on the complete enhanced dataset
    """
    
    def __init__(self, input_dim, encoding_dims=[64, 32, 16], dropout_rate=0.2):
        super(EnhancedAutoEncoder, self).__init__()
        
        self.input_dim = input_dim
        self.encoding_dims = encoding_dims
        self.dropout_rate = dropout_rate
        
        # Build encoder layers dynamically
        encoder_layers = []
        prev_dim = input_dim
        
        for i, dim in enumerate(encoding_dims):
            encoder_layers.extend([
                nn.Linear(prev_dim, dim),
                nn.BatchNorm1d(dim),  # Batch normalization for stability
                nn.ReLU(),
                nn.Dropout(dropout_rate)  # Dropout for regularization
            ])
            prev_dim = dim
        
        # Remove last dropout
        encoder_layers = encoder_layers[:-1]
        self.encoder = nn.Sequential(*encoder_layers)
        
        # Build decoder layers (reverse of encoder)
        decoder_layers = []
        reverse_dims = list(reversed(encoding_dims[:-1])) + [input_dim]
        prev_dim = encoding_dims[-1]
        
        for i, dim in enumerate(reverse_dims):
            if i == len(reverse_dims) - 1:  # Last layer (output)
                decoder_layers.extend([
                    nn.Linear(prev_dim, dim),
                    nn.Sigmoid()  # Sigmoid for normalized output
                ])
            else:
                decoder_layers.extend([
                    nn.Linear(prev_dim, dim),
                    nn.BatchNorm1d(dim),
                    nn.ReLU(),
                    nn.Dropout(dropout_rate)
                ])
            prev_dim = dim
        
        self.decoder = nn.Sequential(*decoder_layers)
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Initialize weights using Xavier initialization"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, x):
        """Forward pass through encoder and decoder"""
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded
    
    def encode(self, x):
        """Get encoded representation"""
        return self.encoder(x)
    
    def decode(self, encoded):
        """Decode from encoded representation"""
        return self.decoder(encoded)

print("🏗️ Enhanced AutoEncoder architecture defined!")
print("✨ Features:")
print("  • Multi-layer encoder/decoder")
print("  • Batch normalization for training stability")
print("  • Dropout for regularization")
print("  • Xavier weight initialization")
print("  • Sigmoid output for normalized reconstruction")

In [ ]:
# Data Preparation and Training Utilities
class EarlyStopping:
    """Early stopping to prevent overfitting"""
    
    def __init__(self, patience=10, min_delta=1e-6, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
            
        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False
    
    def save_checkpoint(self, model):
        """Save model weights"""
        self.best_weights = model.state_dict().copy()

def prepare_data_for_training(df, test_size=0.2, val_size=0.2, batch_size=128, random_state=42):
    """Prepare data for AutoEncoder training with proper splits"""
    print("🔄 Preparing data for training...")
    
    # Remove non-numeric columns if any
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    X = df[numeric_cols].values
    
    print(f"📊 Using {len(numeric_cols)} numeric features")
    print(f"🔢 Data shape: {X.shape}")
    
    # Handle any remaining NaN values
    if np.isnan(X).any():
        print("⚠️ Found NaN values, filling with column means...")
        X = np.nan_to_num(X, nan=np.nanmean(X, axis=0))
    
    # Split into train, validation, and test
    X_temp, X_test = train_test_split(X, test_size=test_size, random_state=random_state)
    val_size_adjusted = val_size / (1 - test_size)
    X_train, X_val = train_test_split(X_temp, test_size=val_size_adjusted, random_state=random_state)
    
    print(f"✂️ Data splits:")
    print(f"  • Training: {X_train.shape[0]:,} samples ({X_train.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"  • Validation: {X_val.shape[0]:,} samples ({X_val.shape[0]/X.shape[0]*100:.1f}%)")
    print(f"  • Test: {X_test.shape[0]:,} samples ({X_test.shape[0]/X.shape[0]*100:.1f}%)")
    
    # Normalize data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)
    
    # Convert to PyTorch tensors
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    train_tensor = torch.FloatTensor(X_train_scaled).to(device)
    val_tensor = torch.FloatTensor(X_val_scaled).to(device)
    test_tensor = torch.FloatTensor(X_test_scaled).to(device)
    
    # Create data loaders
    train_dataset = TensorDataset(train_tensor, train_tensor)
    val_dataset = TensorDataset(val_tensor, val_tensor)
    test_dataset = TensorDataset(test_tensor, test_tensor)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    print(f"🚀 Data moved to device: {device}")
    print("✅ Data preparation complete!")
    
    return {
        'train_loader': train_loader,
        'val_loader': val_loader, 
        'test_loader': test_loader,
        'scaler': scaler,
        'device': device,
        'input_dim': X_train_scaled.shape[1]
    }

print("⏱️ Early stopping mechanism ready!")
print("🔧 Data preparation utilities defined!")

In [ ]:
# Prepare Data for Training
data_info = prepare_data_for_training(df_enhanced_numeric, batch_size=256)

# Training Function
def train_enhanced_autoencoder(data_info, epochs=100, learning_rate=0.001, 
                             weight_decay=1e-5, patience=15, encoding_dims=[64, 32, 16]):
    """Train enhanced AutoEncoder with all improvements"""
    device = data_info['device']
    input_dim = data_info['input_dim']
    
    print(f"🚀 Starting enhanced AutoEncoder training")
    print(f"📊 Input dimension: {input_dim}")
    print(f"🏗️ Architecture: {input_dim} → {' → '.join(map(str, encoding_dims))} → {' → '.join(map(str, reversed(encoding_dims)))} → {input_dim}")
    print(f"⏱️ Max epochs: {epochs}")
    print(f"🎯 Device: {device}")
    print()
    
    # Initialize model
    model = EnhancedAutoEncoder(
        input_dim=input_dim, 
        encoding_dims=encoding_dims,
        dropout_rate=0.2
    ).to(device)
    
    # Loss function and optimizer with weight decay
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )
    
    # Early stopping
    early_stopping = EarlyStopping(patience=patience, min_delta=1e-6)
    
    # Training history
    history = {'train_loss': [], 'val_loss': [], 'lr': []}
    
    print("🔄 Training started...")
    epoch_pbar = tqdm(range(epochs), desc="Training", unit="epoch")
    
    for epoch in epoch_pbar:
        # Training phase
        model.train()
        train_loss = 0.0
        train_batches = 0
        
        for batch_x, _ in data_info['train_loader']:
            optimizer.zero_grad()
            reconstructed, encoded = model(batch_x)
            loss = criterion(reconstructed, batch_x)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_batches += 1
        
        avg_train_loss = train_loss / train_batches
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_batches = 0
        
        with torch.no_grad():
            for batch_x, _ in data_info['val_loader']:
                reconstructed, _ = model(batch_x)
                loss = criterion(reconstructed, batch_x)
                val_loss += loss.item()
                val_batches += 1
        
        avg_val_loss = val_loss / val_batches
        scheduler.step(avg_val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['lr'].append(current_lr)
        
        # Update progress bar
        epoch_pbar.set_postfix({
            'train_loss': f'{avg_train_loss:.6f}',
            'val_loss': f'{avg_val_loss:.6f}',
            'lr': f'{current_lr:.2e}'
        })
        
        # Early stopping check
        if early_stopping(avg_val_loss, model):
            print(f"\n⏹️ Early stopping triggered at epoch {epoch+1}")
            break
    
    epoch_pbar.close()
    print("\n✅ Training completed!")
    return model, history

# Train the model
print("🎯 Training Enhanced AutoEncoder on Complete Feature Dataset")
print("=" * 70)

model, history = train_enhanced_autoencoder(
    data_info, 
    epochs=100,
    learning_rate=0.001,
    weight_decay=1e-5,
    patience=15,
    encoding_dims=[64, 32, 16]
)